# Regression_Adam

## SGDM
SGDM即为SGD with momentum，它加入了动量机制，1986年提出。

![title](attachment/1-5.png)

避免落入局部最小值。

![title](attachment/1-8.png)

缺点：没有学习率控制



In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/train.csv', encoding = 'big5')
data.head(10)

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014/1/1,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,...,11,11,22,28,19,12,8.1,7,6.9,6
6,2014/1/1,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014/1/1,豐原,O3,16,30,27,23,24,28,24,...,65,64,51,34,33,34,37,38,38,36
8,2014/1/1,豐原,PM10,56,50,48,35,25,12,4,...,52,51,66,85,85,63,46,36,42,42
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13


In [2]:
def to_rawdata(data, n = 3):
    raw_data = data.iloc[:,n:]
    raw_data[raw_data == 'NR'] = 0
    raw_data = raw_data.astype(float)
    return raw_data

raw_data = to_rawdata(data)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)


In [3]:
raw_data = raw_data.values

def classfi(raw_data):
    X = []
    y = []
    for i in range(0, 4320, 18):
        for j in range(0, 15):
            X.append(raw_data[i:i+18,j:j+9])
            y.append(raw_data[i+9,j+9])
    return X, y

X, y = classfi(raw_data)

In [4]:
X = np.array(X)

## 用前九天的PM2.5来预测第十天的PM2.5
### SGDM

In [5]:
X_pm25 = []
for i in range(X.shape[0]):
    X_pm25.append(X[i][9])
    
X_pm25 = np.array(X_pm25)

In [6]:
def x_pre(x):
    b0 = np.ones(x.shape[0]).reshape(-1, 1)
    return np.hstack([b0,x])

def gradient_descent_sgdm(X, y, n_iters = 10e6, p = 0.9, batch_size = 50, eta = 0.1):  

    def dJ(beta, X_b, y):
        return X_b.T.dot(X_b.dot(beta) - y) * 2 / len(X_b)
    
    X = x_pre(X)
    y = np.array(y)
    initial_beta = np.random.rand(X.shape[1])
     
    i_iter = 0
    movement = 0
    beta = initial_beta
    m = X.shape[0]
    
    
    while i_iter < n_iters:
        rand_i = np.random.permutation(m)
        for i in range(m//batch_size):
            gradient = dJ(beta, X[rand_i][i*batch_size:(i+1)*batch_size], y[rand_i][i*batch_size:(i+1)*batch_size])
            movement = p*movement + (1-p)*gradient
            beta = beta - eta * movement
            i_iter += 1
    
    
    return beta[0], beta[1:]

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

normal = StandardScaler()
normal.fit(X_pm25)
X_pm25 = normal.transform(X_pm25)

X_train, X_test, y_train, y_test = train_test_split(X_pm25, y, test_size = 0.8)

In [8]:
%%time
intercept_, coef_ = gradient_descent_sgdm(X_train, y_train, n_iters = 100000, 
                             batch_size = 100, p = 0.999)
intercept_

CPU times: user 2.86 s, sys: 54.7 ms, total: 2.91 s
Wall time: 2.93 s


24.11430357720519

In [9]:
coef_

array([  0.03474067,  -1.26759358,   4.56770387,  -5.07726794,
         0.33512582,   8.72740603, -11.25090417,   2.07081411,
        17.45832668])

In [10]:
from sklearn.metrics import mean_squared_error

y_predict = X_test.dot(coef_) + intercept_
mean_squared_error(y_test, y_predict)

44.11043389057022

##### Predict

In [11]:
test_data = pd.read_csv('data/test.csv', encoding = 'big5', header = None )
test_data.head(10)

raw_test = to_rawdata(test_data, n = 2)
raw_test = raw_test.values

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)


In [12]:
X_predict = []
for i in range(0, raw_test.shape[0], 18):
    X_predict.append(raw_test[i+9,:])
        
X_predict = np.array(X_test)
X_predict_normal = normal.transform(X_predict)

In [13]:
y_predict_sgdm = intercept_ + X_predict_normal.dot(coef_)

predict_answer = []

for i in range(y_predict_sgdm.shape[0]):
    predict_answer.append([f'id_{i}', y_predict_sgdm[i]])  

In [14]:
pd_predict_answer = pd.DataFrame(predict_answer, columns = ['id', 'PM2.5'] )
pd_predict_answer.head()

,id,PM2.5
0,id_0,2.428522
1,id_1,2.076697
2,id_2,2.283388
3,id_3,2.154352
4,id_4,2.779459


In [15]:
pd_predict_answer.to_csv('predict_answer/sgdm.csv', index = False)

## RMSProp

![title](attachment/1-6.png)

降低学习率，类比Adagrad




In [16]:
def gradient_descent_RMSProp(X, y, n_iters = 10e6, alpha = 0.999, eta = 0.1):  

    def dJ(beta, X_b, y):
        return X_b.T.dot(X_b.dot(beta) - y) * 2 / len(X_b)
    
    X = x_pre(X)
    y = np.array(y)
    initial_beta = np.random.rand(X.shape[1])
     
    beta = initial_beta
    i_iter = 0    
    v = dJ(beta, X, y) ** 2   
    
    while i_iter < n_iters:
        gradient = dJ(beta, X, y)
        beta = beta - (eta/np.sqrt(v))*gradient
        v = alpha*v + (1-alpha)*(gradient ** 2)
        i_iter += 1
    
    return beta[0], beta[1:]

In [17]:
%%time
intercept_, coef_ = gradient_descent_RMSProp(X_train, y_train, n_iters = 100000)
intercept_

CPU times: user 9.31 s, sys: 75.2 ms, total: 9.39 s
Wall time: 2.39 s


24.105503003403104

In [18]:
coef_

array([  0.05616961,  -1.22557633,   4.63039035,  -5.06787942,
         0.35761251,   8.79519351, -11.20820286,   2.0679212 ,
        17.49149943])

In [19]:
y_predict = X_test.dot(coef_) + intercept_
mean_squared_error(y_test, y_predict)

44.123733213307816

In [20]:
y_predict_RMSProp = intercept_ + X_predict_normal.dot(coef_)

predict_answer = []

for i in range(y_predict_RMSProp.shape[0]):
    predict_answer.append([f'id_{i}', y_predict_RMSProp[i]]) 

pd_predict_answer = pd.DataFrame(predict_answer, columns = ['id', 'PM2.5'] )
pd_predict_answer.head()

,id,PM2.5
0,id_0,2.043674
1,id_1,1.668277
2,id_2,1.916104
3,id_3,1.750837
4,id_4,2.390916


In [21]:
pd_predict_answer.to_csv('predict_answer/RMSProp.csv', index = False)

## Adam
![title](attachment/1-7.png)




In [22]:
def gradient_descent_adam(X, y, n_iters = 10e6, beta_1 = 0.9, 
                          beta_2 = 0.999, eta = 0.1, epsilon = 10e-8,  
                          batch_size = 50, t = 2):  

    def dJ(beta, X_b, y):
        return X_b.T.dot(X_b.dot(beta) - y) * 2 / len(X_b)
    
    X = x_pre(X)
    y = np.array(y)
    initial_beta = np.random.rand(X.shape[1])
     
    beta = initial_beta
    i_iter = 0    
    v = dJ(beta, X, y) ** 2      
    movement = 0
    m = X.shape[0]
    
    while i_iter < n_iters:
        rand_i = np.random.permutation(m)
        for i in range(m//batch_size):
            gradient = dJ(beta, X[rand_i][i*batch_size:(i+1)*batch_size], y[rand_i][i*batch_size:(i+1)*batch_size])
            movement = beta_1*movement + (1-beta_1)*gradient
            movement_hat = movement/(1-(beta_1 ** t))
            v_hat = v/(1-(beta_2)**t)
            beta = beta - eta/(np.sqrt(v_hat)+epsilon)*movement_hat
            v = beta_2*v + (1-beta_2)*(gradient**2)
            i_iter += 1      
            
    return beta[0], beta[1:]

In [23]:
%%time
intercept_, coef_ = gradient_descent_adam(X_train, y_train, n_iters = 100000)
intercept_

CPU times: user 4.24 s, sys: 96.9 ms, total: 4.34 s
Wall time: 3.84 s


24.061183249707295

In [24]:
coef_

array([  0.04170037,  -1.18900037,   4.64822607,  -5.06902603,
         0.36212723,   8.78854534, -11.22594322,   2.01267081,
        17.59143014])

In [25]:
y_predict = X_test.dot(coef_) + intercept_
mean_squared_error(y_test, y_predict)

44.09283352710668

In [26]:
y_predict_adam = intercept_ + X_predict_normal.dot(coef_)

predict_answer = []

for i in range(y_predict_adam.shape[0]):
    predict_answer.append([f'id_{i}', y_predict_adam[i]]) 

pd_predict_answer = pd.DataFrame(predict_answer, columns = ['id', 'PM2.5'] )
pd_predict_answer.head()

,id,PM2.5
0,id_0,1.913248
1,id_1,1.537567
2,id_2,1.787429
3,id_3,1.618432
4,id_4,2.261727


In [27]:
pd_predict_answer.to_csv('predict_answer/adam.csv', index = False)

## Application

![title](attachment/1-9.png)